In [9]:
%pip install transformers datasets torch gradio sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import pipeline


/Users/albertwelong/cognizant_genai/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/albertwelong/cognizant_genai/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-08 17:41:24.044682: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Use a pipeline as a high-level helper
pipe = pipeline("text-generation", model="mistralai/Mistral-7B-v0.1")



Loading checkpoint shards: 100%|██████████| 2/2 [04:42<00:00, 141.25s/it]
Device set to use cpu


In [10]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Load the FAQ dataset
faq_data = pd.read_csv("faq_dataset.csv")

# Extract questions, answers, and categories
faq_questions = faq_data['question'].tolist()
faq_answers = faq_data['answer'].tolist()
# faq_categories = faq_data['category'].tolist()

# Load a pre-trained BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Compute embeddings for FAQ questions
faq_embeddings = model.encode(faq_questions, convert_to_tensor=True)

def get_faq_answer(user_input, threshold=0.5):  # Set a similarity threshold
    input_embedding = model.encode(user_input, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(input_embedding, faq_embeddings)
    
    # Find the most similar question
    best_match_idx = similarities.argmax()
    best_match_score = similarities[0][best_match_idx].item()

    # If similarity score is too low, return a general response
    if best_match_score < threshold:
        return "I'm not sure about that, but I can help with FAQs! Ask me about orders, refunds, or other support."
        # response = pipe(user_input, max_length=200, num_return_sequences=1, temperature=0.7, do_sample=True)
        # return response[0]["generated_text"]

    return faq_answers[best_match_idx]

# Example:
print(get_faq_answer("Do you have any promotions available?"))
print(get_faq_answer("What are your business hours?"))  # Expected: FAQ response
print(get_faq_answer("Tell me about your refund policy"))  # Expected: FAQ response
print(get_faq_answer("What is your return policy?"))  # Expected: FAQ response
print(get_faq_answer("How can I track my order?"))  # Expected: FAQ response
print(get_faq_answer("Do you offer discounts?"))  # Expected: FAQ response
print(get_faq_answer("Name a city in France"))  # Expected: General model response
print(get_faq_answer("Tell me a joke."))  # Expected: General model response




Yes, we offer discounts through seasonal promotions and occasional sales. Subscribe to our newsletter for updates.
We are open from 9 AM to 6 PM, Monday through Friday.
Our refund policy allows returns within 30 days of purchase with proof of purchase.
Our refund policy allows returns within 30 days of purchase with proof of purchase.
You can track your order by visiting the 'Order Tracking' section on our website, using your order number.
Yes, we offer discounts through seasonal promotions and occasional sales. Subscribe to our newsletter for updates.
I'm not sure about that, but I can help with FAQs! Ask me about orders, refunds, or other support.
I'm not sure about that, but I can help with FAQs! Ask me about orders, refunds, or other support.


In [11]:
model.save("./faq_model")

In [12]:
import torch

# Save embeddings
torch.save(faq_embeddings, "faq_embeddings.pt")

# Save FAQ data as a CSV or JSON (to load it later)
faq_data.to_csv("faq_data.csv", index=False)


# Short Evaluation for Future Improvements

## Due to the interest of computing capability and time, I decided to tackle User Story 1 with a simpler and more direct approach.

1. I generated my own FAQ dataset that can answer basic FAQs.
2. The response generated follows the FAQ dataset that has been made
3. The model is fine-tuned to find keywords that can predict what the user input look for, and then find the response on the FAQ answer column.
4. For future iteration, it is in the best interest to also train a general pre-trained model to answer non-relevant questions like a smart chatbot assistant. But for the interest of this project which focuses on creating a personalized chatbot for businesses, non-relevant questions will be responded by a predetermined response saying "I'm not sure about that, but I can help with FAQs! Ask me about orders, refunds, or other support."